In [1]:
#initialize everything

import pandas as pd
import re
from ftfy import fix_text
from unidecode import unidecode

# Import data from json file
df = pd.read_json('gg2013.json')

# Import list of actors and movies from IMDB (uncomment when actually used so it doesn't take forever to run)
# actor_df = pd.read_csv('data\\name.basics.tsv', sep='\t')
# movie_df = pd.read_csv('data\\title.basics.tsv', sep='\t')

# actors = actor_df['primaryName']
# print(actors)

# Get only the body of the tweets (ignore username, etc.) and clean it up
text = df['text']
text = text.tolist()

for tweet in text:
    tweet = fix_text(tweet)
    tweet = unidecode(tweet)
    tweet = " ".join(tweet.split())

In [24]:
#define regex + candidate lists for each topic

# Regular Expressions for winners
#I wanted to include [?!(should)] here but if I do then the # of usable tweets drops from 3119 -> 696
win_exprs = ['(wins|Wins|WINS|receiv(es|ed)|won)(?= best| Best| BEST)', '(best(.+)|Best(.+)|BEST(.+))(?= goes to| Goes To| GOES TO)']

# Regular Expressiosn for hosts
host_exprs = ['(hosts?|Hosts?|HOSTS?|hosted by)[?!(will|should)]']

# Regular Expression for nominees
#changed the order of regex in case that matters
nom_exprs = ['[?!(pretend|fake|was not|is not)](nominees?|Nominees?|NOMINEES?|nominated?)']

# Regular Expression for presenters
presenter_exprs = ['(presenters?|Presenters?|PRESENTERS?|presented by|present(ed|s|ing))']

# Intialize Candidate Lists
win_candidates = []
win_count = -1

category_candidates = []
category_count = -1

host_candidates_left = []
host_count_left = -1

host_candidates_right = []
host_count_right = -1

nom_candidates_left = []
nom_count_left = -1

nom_candidates_right = []
nom_count_right = -1

presenter_candidates_left = []
presenter_count_left = -1

presenter_candidates_right = []
presenter_count_right = -1

In [3]:
#split winners and categories

test = []

for tweet in text:
    for expr in win_exprs:
        # example regex use
        var = re.findall(r"(.+) " + expr + " (.+)", tweet)
        if var:
            win_candidates.append([])
            win_count += 1
            category_candidates.append([])
            category_count += 1
            test.append(var)
            before = var[0][0].rsplit(None, var[0][0].count(' '))
            for i in range(0, len(before)):
                new_candidate = before[-1]
                for j in range(0, i):
                    new_candidate = before[len(before) - j - 2] + " " + new_candidate
                win_candidates[win_count].append(new_candidate)
            after = var[0][2].rsplit(None, var[0][2].count(' '))
            for i in range(0, len(after)):
                new_candidate = after[0]
                for j in range(0, i):
                    new_candidate = new_candidate + " " + after[1 + j]
                category_candidates[category_count].append(new_candidate)

In [26]:
#split host tweets left and right --> need to combine these lists after?

test = []

for tweet in text:
    for expr in host_exprs:
        # example regex use
        var = re.findall(r"(.+) " + expr + " (.+)", tweet)
        if var:
            host_candidates_left.append([])
            host_count_left += 1
            host_candidates_right.append([])
            host_count_right += 1
            test.append(var)
            before = var[0][0].rsplit(None, var[0][0].count(' '))
            for i in range(0, len(before)):
                new_candidate = before[-1]
                for j in range(0, i):
                    new_candidate = before[len(before) - j - 2] + " " + new_candidate
                host_candidates_left[host_count_left].append(new_candidate)
            after = var[0][2].rsplit(None, var[0][2].count(' '))
            for i in range(0, len(after)):
                new_candidate = after[0]
                for j in range(0, i):
                    new_candidate = new_candidate + " " + after[1 + j]
                host_candidates_right[host_count_right].append(new_candidate)

#combine left and right lists
host_candidates = host_candidates_left + host_candidates_right

In [13]:
#split nominee tweets left and right

test = []

for tweet in text:
    for expr in nom_exprs:
        # example regex use
        var = re.findall(r"(.+) " + expr + " (.+)", tweet)
        if var:
            nom_candidates_left.append([])
            nom_count_left += 1
            nom_candidates_right.append([])
            nom_count_right += 1
            test.append(var)
            before = var[0][0].rsplit(None, var[0][0].count(' '))
            for i in range(0, len(before)):
                new_candidate = before[-1]
                for j in range(0, i):
                    new_candidate = before[len(before) - j - 2] + " " + new_candidate
                nom_candidates_left[nom_count_left].append(new_candidate)
            after = var[0][2].rsplit(None, var[0][2].count(' '))
            for i in range(0, len(after)):
                new_candidate = after[0]
                for j in range(0, i):
                    new_candidate = new_candidate + " " + after[1 + j]
                nom_candidates_right[nom_count_right].append(new_candidate)

#combine left and right lists
nom_candidates = nom_candidates_left + nom_candidates_right

In [14]:
#split presenter tweets left and right --> need to combine these lists after?

test = []

for tweet in text:
    for expr in presenter_exprs:
        # example regex use
        var = re.findall(r"(.+) " + expr + " (.+)", tweet)
        if var:
            presenter_candidates_left.append([])
            presenter_count_left += 1
            presenter_candidates_right.append([])
            presenter_count_right += 1
            test.append(var)
            before = var[0][0].rsplit(None, var[0][0].count(' '))
            for i in range(0, len(before)):
                new_candidate = before[-1]
                for j in range(0, i):
                    new_candidate = before[len(before) - j - 2] + " " + new_candidate
                presenter_candidates_left[presenter_count_left].append(new_candidate)
            after = var[0][2].rsplit(None, var[0][2].count(' '))
            for i in range(0, len(after)):
                new_candidate = after[0]
                for j in range(0, i):
                    new_candidate = new_candidate + " " + after[1 + j]
                presenter_candidates_right[presenter_count_right].append(new_candidate)
                
#combine left and right lists
presenter_candidates = presenter_candidates_left + presenter_candidates_right

In [29]:
#apply quality filter on split tweets (gets rid of tweets containing @'s)

#tried making a list of multiple things to filter but it retains http's anyway and also
#makes duplicate entries:

#quality_filter = ['@','http'] 
#for j in range(len(quality_filter):
    #pattern = re.compile('[{}]'.format(quality_filter))
    #for i in range(len(win_candidates)):
        #filtered_winners += list(filter(lambda x: not pattern.search(x), win_candidates[i]))

quality_filter = '@'

pattern = re.compile('[{}]'.format(quality_filter))

filtered_winners = []
for i in range(len(win_candidates)):
    filtered_winners += list(filter(lambda x: not pattern.search(x), win_candidates[i]))
    
filtered_categories = []
for i in range(len(category_candidates)):
    filtered_categories += list(filter(lambda x: not pattern.search(x), category_candidates[i]))
    
filtered_hosts = []
for i in range(len(host_candidates)):
    filtered_hosts += list(filter(lambda x: not pattern.search(x), host_candidates[i]))
    
filtered_nominees = []
for i in range(len(nom_candidates)):
    filtered_nominees += list(filter(lambda x: not pattern.search(x), nom_candidates[i]))
    
filtered_presenters = []
for i in range(len(presenter_candidates)):
    filtered_presenters += list(filter(lambda x: not pattern.search(x), presenter_candidates[i]))

In [ ]:
#print(filtered_winners)